In [1]:
import numpy as np
import pandas as pd
import nltk
import re
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from scipy import spatial
import networkx as nx
import warnings
warnings.filterwarnings(action='ignore')

def summarize(text,ratio):
  # split the the text in the articles into sentences
  sentences=sent_tokenize(text)
  sentences_clean=[re.sub(r'[^\w\s]','',sentence.lower()) for sentence in sentences]
  stop_words = stopwords.words('english')
  sentence_tokens=[[words for words in sentence.split(' ') if words not in stop_words] for sentence in sentences_clean] 
  #w2v=Word2Vec(sentence_tokens,size=4,min_count=1,iter=1000)
  w2v=Word2Vec(sentence_tokens,vector_size=4,min_count=1,epochs=1000)
  sentence_embeddings=[[w2v[word][0] for word in words] for words in sentence_tokens]
  max_len=max([len(tokens) for tokens in sentence_tokens])
  sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]

  similarity_matrix = np.zeros([len(sentence_tokens), len(sentence_tokens)])
  for i,row_embedding in enumerate(sentence_embeddings):
      for j,column_embedding in enumerate(sentence_embeddings):
          similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
  nx_graph = nx.from_numpy_array(similarity_matrix)
  scores = nx.pagerank(nx_graph)
  #scores = nx.pagerank(nx_graph, max_iter=600)
  #scores = nx.pagerank(nx_graph, tol=1.0e-9, max_iter=2000)
  #storing the sentences in decreasing order of score
  ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)

  # Generate summary
  text_summary=""
  for i in range(int(ratio)):
    text_summary = text_summary+ " " + str(ranked_sentences[i][1])
  return text_summary